In [ ]:
from nltk import CFG
G = CFG.fromstring("""
    S -> A B C | D E
    A -> 2 B 3 | ε
    B -> BP
    BP -> 4 C 5 BP | ε
    C -> 6 A B | ε
    D -> 1 A E | B
    E -> 3

""")

In [ ]:
def obtener_simbolos(grammar):
    """
    Función que devuelve una lista de los símbolos terminales y no terminales de una gramática CFG dada.
    """
    simbolos = set()
    for production in grammar.productions():
        simbolos.add(production.lhs())  # agregamos el símbolo no terminal de la regla
        for symbol in production.rhs(): # agregamos los símbolos terminales o no terminales de la regla
            simbolos.add(symbol)
    return list(simbolos)

def obtener_no_terminales(grammar):
    """
    Función que devuelve una lista de los símbolos no terminales de una gramática CFG dada.
    """
    no_terminales = set()
    for production in grammar.productions():
        no_terminales.add(production.lhs())  # agregamos el símbolo no terminal de la regla
    return list(no_terminales)

def obtener_terminales(grammar):
    """
    Función que devuelve una lista de los símbolos terminales de una gramática CFG dada.
    """
    no_terminales = obtener_no_terminales(grammar)
    simbolos = set()
    for production in grammar.productions():
        for symbol in production.rhs():
            if symbol not in no_terminales:
                simbolos.add(symbol)
    return list(simbolos)



print(obtener_simbolos(G))
print(obtener_no_terminales(G))
print(obtener_terminales(G))


[A, D, S, 1, 4, E, 6, C, BP, ε, 3, 2, B, 5]
[A, D, S, E, C, BP, B]
[1, 4, 6, ε, 3, 2, 5]


In [ ]:
from nltk import nonterminals

In [ ]:
def left_factor(grammar):
    # Identificar símbolos no terminales en la gramática
    non_terminals = nonterminals(",".join(sorted(set(str(p.lhs()) for p in grammar.productions()))))


    # Para cada símbolo no terminal, identificar las producciones con prefijo común
    for A in non_terminals:
        productions = grammar.productions(A)
        prefixes = {}
        for prod in productions:
            if prod.rhs()[0] not in prefixes:

                prefixes[prod.rhs()[0]] = []

            prefixes[prod.rhs()[0]].append(prod.rhs()[1:])

        # Si existen producciones con prefijo común, factorizar la gramática
        for prefix, suffixes in prefixes.items():
            if len(suffixes) > 1:
                A_ = grammar.new_nonterminal(A.symbol+"'")
                grammar.add_nonterminal(A_)
                grammar.remove_productions(A, lambda prod: prod.rhs[0] == prefix)
                for suffix in suffixes:
                    grammar.add_production(A_, suffix)
                grammar.add_production(A, [prefix, A_])

grammar = CFG.fromstring("""
    S -> A B C | D E
    A -> 2 B 3 | ε
    B -> 4 C 5 B | ε
    C -> 6 A B | ε
    D -> 1 A E | B
    E -> 3
""")
left_factor(grammar)
print(grammar)

Grammar with 12 productions (start state = S)
    S -> A B C
    S -> D E
    A -> 2 B 3
    A -> ε
    B -> Z
    Z -> 4 C 5 Z
    Z -> ε
    C -> 6 A B
    C -> ε
    D -> 1 A E
    D -> B
    E -> 3


In [ ]:
def calcular_primeros_gramatica(gramatica):
    primeros = {}
    for no_terminal in gramatica:
        primeros[no_terminal] = calcular_primeros(no_terminal, gramatica)
    return primeros

def calcular_primeros(simbolo, gramatica):
    primeros = set()
    for produccion in gramatica[simbolo]:
        if produccion[0].isupper(): # si la producción comienza con un no-terminal
            primeros_produccion = calcular_primeros(produccion[0], gramatica)
            primeros |= primeros_produccion
            if 'ε' in primeros_produccion:
                i = 1
                while i < len(produccion) and 'ε' in primeros_produccion:
                    primeros_produccion = calcular_primeros(produccion[i], gramatica)
                    primeros |= primeros_produccion - {'ε'}
                    i += 1
                if i == len(produccion) and 'ε' in primeros_produccion:
                    primeros.add('ε')
        else: # si la producción comienza con un terminal
            primeros.add(produccion[0])
    return primeros


In [ ]:
gramatica = {'S': ['ABC', 'DE'], 'A': ['2B3','ε'], 'B': ['4C5B','ε'], 'C': ['6AB','ε'], 'D': ['1AE','B'], 'E': ['3', 'ε']}
primeros = calcular_primeros_gramatica(gramatica)
print(primeros)


{'S': {'4', '6', '3', '1', '2', 'ε'}, 'A': {'2', 'ε'}, 'B': {'ε', '4'}, 'C': {'6', 'ε'}, 'D': {'4', '1', 'ε'}, 'E': {'3', 'ε'}}
